Please follow the rules/instructions for making the assignments. This means, for every exercise:

- Make sure the code is **well-documented**
- Update the README file, found in the same directory as the assignment, this means:
    - Describe the objective of the assignment
    - Provide instructions on how to run your code
    - Include any other relevant notes or dependencies

Also, please be mindful of the rules for the use of Generative AI:
- You may use generative AI tools (e.g., ChatGPT, GitHub Copilot, etc.) in your assignments under the following conditions:
    - Clearly indicate where and how you used generative AI
    - Save and submit the AI conversation alongside your assignment (upload the transcript alongside your assignment)
    - Remember: Using AI tools should support your learning, not replace it. Make sure the work you submit reflects your group’s understanding. All group members are still responsible for understanding and being able to explain all submitted content.

***The name of the group member responsible for each assignment is written behind the exercise number.***

In [1]:
import numpy, cobra, sys, pathlib
print("Python:", sys.version.split()[0])
print("NumPy:", numpy.__version__)
print("COBRA:", cobra.__version__)
print("CWD:", pathlib.Path.cwd())

Python: 3.12.1
NumPy: 2.2.6
COBRA: 0.29.1
CWD: /Users/elevels/Documents/UM/Bachelor/Year 3/Period 1/Multi-Scale Modelling of Biological Systems/Assignments/Group_10_Biological_Systems/Assignment1


# Exercise 1 - Evi Levels

**a)** When looking at the ESCHER map of the E. coli core model, we do not observe the same pattern as we did for reaction fluxes in a linear pathway. Previously, fluxes in a linear pathway were equal due to mass balance constraints, but for maximal reaction activities, the arrows representing sequential reactions often have different colors or shades, indicating different activity levels. For example, the arrow for PFK has a lighter shade of green than the arrow for FBA, indicating a lower maximal reaction acitivity (in mmol/gDW/h) for PFK compared to FBA. Thus, we can conclude that the maximal reaction activities are not equal to each other in a linear pathway. A reason for this could be that the maximal reaction activities show the estimated maximal activity for each reaction individually, instead of the actual fluxes that are constrained by mass balance. 

**b)** Two examples of reactions that have no maximal reaction activity data are *EX_h2o_e* and *EX_co2_e*, which are H2O exchange and CO2 exchange reactions, respectively. Gene expression-derived data would not be applicable because exchange reactions are not physical reactions, but they are artificial reactions describing the system boundaries. Since there are no genes or enzymes involved, there is no gene expression-derived data associated with them. 

# Exercise 2 - Evi Levels

In [2]:
import pandas as pd

In [3]:
# Loading the model
model = cobra.io.load_json_model('e_coli_core.json')

In [4]:
# Taking a look at the model
model

Name,e_coli_core
Memory address,15ec843b0
Number of metabolites,72
Number of reactions,95
Number of genes,137
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"extracellular space, cytosol"


In [5]:
# Loading the gene expression data 
gene_expression_data = pd.read_csv("e_coli_core_expression.csv")
gene_expression_data.head()

,# Reaction ID,reaction activity [mmol/gDW/h]
0,PFK,12.12
1,PFL,1.00
2,PGI,13.12
3,PGK,23.13
4,PGL,8.12


In [6]:
# renaming the columns for easier usage
gene_expression_data.columns = ['reaction_id', 'max_activity']
gene_expression_data.head()

,reaction_id,max_activity
0,PFK,12.12
1,PFL,1.00
2,PGI,13.12
3,PGK,23.13
4,PGL,8.12


For the ATPM energy maintenance reaction, we leave the lower flux as-is, since it describes cellular energy requirements separate from any maximal reaction activities. 

Then we have to, for reversible reactions, set the lower and upper flux bound to -value and +value, respectively (“value” as the maximal reaction activity). We know a reaction is reversible if the lower bound is < 0 and the upper bound is > 0.

Next we have to, for irreversible reactions, only set the maximal flux bound to value and leave the minimal flux bound at zero. A reaction is irreversible if we have lower bound >= 0 AND upper bound > 0. 

Technically, we could also have another situation for when a reaction is irreversible. This is when we have lower bound < 0 AND upper bound <= 0. However, when looking at the assignment, I suppose we have to ignore this case (at least for now).

We leave the default constraints for reactions without data.

In [7]:
for reaction in model.reactions:
    # Leave ATPM lower flux bound as-is
    if reaction.id == "ATPM":
        continue

    # Get maximal reaction activity
    value = gene_expression_data[gene_expression_data['reaction_id'] == reaction.id]['max_activity']

    # Check if non-empty, so to see if the reaction is in gene_expression_data
    if value.empty:
        # Leave default constraints for reactions without data
        continue
    
    value = value.iloc[0]
    
    # search for reversible reactions
    if reaction.lower_bound < 0 and reaction.upper_bound > 0: 
        # Set lower and upper bound to -value and +value respectively
        reaction.lower_bound = -value
        reaction.upper_bound = value
        
    # search for irreversible reactions
    elif reaction.lower_bound >= 0 and reaction.upper_bound > 0:
        # Leave lower bound at zero
        reaction.lower_bound = 0
        # Set upper bound to value
        reaction.upper_bound = value

Lastly, we take a look at the glucose exchange reaction, *EX_glc__D_e*. We have to remove the maximal absolute flux bound
and use the high absolute default bound instead.

In [8]:
model.reactions.EX_glc__D_e.lower_bound = -1000
model.reactions.EX_glc__D_e.upper_bound = 1000

Printing a table listing each reaction’s lower and upper flux bound:

In [9]:
print(f"{'Reaction ID':<25} | {'Lower bound':<12} | {'Upper bound':<12}\n")
for reaction in model.reactions:
    print(f"{reaction.id:<25} | {reaction.lower_bound:<12} | {reaction.upper_bound:<12}")

Reaction ID               | Lower bound  | Upper bound 

PFK                       | 0            | 12.12       
PFL                       | 0            | 1.0         
PGI                       | -13.12       | 13.12       
PGK                       | -23.13       | 23.13       
PGL                       | 0            | 8.12        
ACALD                     | -1.16        | 1.16        
AKGt2r                    | -3.1         | 3.1         
PGM                       | -20.01       | 20.01       
PIt2r                     | -6.03        | 6.03        
ALCD2x                    | -9.01        | 9.01        
ACALDt                    | -2.29        | 2.29        
ACKr                      | -1.19        | 1.19        
PPC                       | 0            | 2.56        
ACONTa                    | -25.35       | 25.35       
ACONTb                    | -25.35       | 25.35       
ATPM                      | 8.39         | 1000.0      
PPCK                      | 0            | 25.2

# Exercise 3 - Evi Levels

**a)**

FVA for all reactions:

In [10]:
FVA_results = cobra.flux_analysis.variability.flux_variability_analysis(
    model = model, 
    reaction_list = model.reactions, 
    loopless = False, 
    fraction_of_optimum = 0.0
)    

Printing resulting minimal and maximal fluxes per reaction:

In [11]:
print(f"{'Reaction ID':<25} | {'Minimal flux':<25} | {'Maximal flux':<25}\n")
for reaction_id, row in FVA_results.iterrows():
    print(f"{reaction_id:<25} | {row['minimum']:<25} | {row['maximum']:<25}")

Reaction ID               | Minimal flux              | Maximal flux             

PFK                       | 0.334677419354839         | 11.15355552574638        
PFL                       | 0.0                       | 1.0                      
PGI                       | -0.08532258064516078      | 9.020350656890784        
PGK                       | -17.596001612302814       | -0.8793548387096762      
PGL                       | 0.0                       | 4.522387640638967        
ACALD                     | -1.16                     | 0.0                      
AKGt2r                    | -3.1                      | 0.0                      
PGM                       | -17.038932281976653       | -0.879354838709677       
PIt2r                     | 4.781534879440234e-15     | 1.783077286504244        
ALCD2x                    | -1.16                     | -7.657268255399939e-17   
ACALDt                    | -1.16                     | 0.0                      
ACKr           

**b)** We identify reactions with gene expression-imposed maximal reaction activities, whose permissible flux range in forward direction is nonzero yet comes out less than its upper flux bound.

In [12]:
count = 0
# Loop through reactions with gene expression-imposed maximal reaction activities
for reaction_id in gene_expression_data['reaction_id']:
    # ignore FORt
    if reaction_id == 'FORt':
        continue 
        
    # Find permissible flux range in forward direction
    permissible_flux_forward = FVA_results.loc[reaction_id, 'maximum']
    # Get reaction
    reaction = model.reactions.get_by_id(reaction_id)
    # Check if permissible_flux_forward is nonzero (so >0, as it is in forward direction) yet comes out less than its upper flux bound
    if permissible_flux_forward > 0 and permissible_flux_forward < reaction.upper_bound:
        print(reaction_id)
        count +=1

print(f"{count} reactions behave this way.")

PFK
PGI
PGL
PIt2r
ACONTa
ACONTb
PPCK
PDH
ADK1
AKGDH
ATPS4r
PTAr
PYK
CO2t
RPE
CS
CYTBD
ENO
SUCDi
TALA
TKT1
TKT2
TPI
FBA
FUM
G6PDH2r
GAPD
GLCpts
GLUDy
GLUN
GLUSy
GND
ICDHyr
ICL
MALS
O2t
36 reactions behave this way.


***NOTE:** We first wrote an initial answer for this exercise ourselves, then asked ChatGPT the same question and used its response to improve our answer.*

Some reactions behave this way because the maximal reaction activities from gene expression are *theoretical* estimates for each reaction individually. In reality, reactions are connected in pathways, and thus the flux through a reaction is constrained by mutliple factors, such as the network-level stoichiometry and mass balance. As a result, even if a reaction could theoretically carry a higher flux, the constraints can limit the actual permissible flux, so it comes out less than the maximal reaction activity.

**c)**

In [13]:
count = 0
for reaction_id, row in FVA_results.iterrows():
    if row['minimum'] > 0:
        count += 1
        print(reaction_id)
        
print(f"{count} reactions have a positive minimal flux in the FVA.")

PFK
PIt2r
ATPM
ATPS4r
CYTBD
ENO
TPI
EX_h_e
EX_h2o_e
FBA
G6PDH2r
GAPD
GLCpts
ICDHyr
NADH16
O2t
16 reactions have a positive minimal flux in the FVA.


***NOTE:** We first wrote an initial answer for this exercise ourselves, then asked ChatGPT the same question and used its response to improve our answer.*

Some reactions have a positive minimal flux in the FVA, meaning they are always active in the forward direction, as their flux can never become zero or go in the other direction. This could indicate that some reactions are stoichiometrically and functionally essential for maintaining the optimal objective flux under the given conditions. They may be responsible for producing metabolites with high demand or metabolites that cannot be produced through alternative pathways. As a result, the network cannot set them to zero without reducing the objective, which explains why their minimal flux remains strictly positive.

# Exercise 4 – Elias Loisel

**a)** Carry out an FBA optimization of biomass and report the maximal biomass production rate.

**b)** Determine the “bottleneck(s)” – i.e. those reaction(s) whose flux reaches their maximum.

**c)** Pick a reaction with maximal reaction activity constraints and zero flux in the optimal solution, and explain from the network context why it carries no flux.


In [14]:
try:
    model.objective = model.reactions.get_by_id("Biomass_Ecoli_core_w_GAM")
except KeyError:
    pass

solution = model.optimize()
print(f"Maximal biomass production rate: {solution.objective_value:.6f} (1/h)")


Maximal biomass production rate: 0.484703 (1/h)


**Answer 4a**  
The maximal biomass production rate with the implemented constraints is **0.484703(1/h)**.


In [15]:
TOL = 1e-6
expr_ids = set(gene_expression_data["reaction_id"].tolist())

def tight_direction(rxn, flux, tol=TOL):
    if flux >= 0 and abs(flux - rxn.upper_bound) <= tol:
        return "upper"
    if flux < 0 and abs(flux - rxn.lower_bound) <= tol:
        return "lower"
    return None

bottlenecks = []
for rxn in model.reactions:
    f = float(solution.fluxes[rxn.id])
    d = tight_direction(rxn, f)
    if d is not None:
        bottlenecks.append({
            "id": rxn.id,
            "name": rxn.name,
            "flux": f,
            "lower": rxn.lower_bound,
            "upper": rxn.upper_bound,
            "direction": d,
            "expr_constrained": (rxn.id in expr_ids),
        })

print(f"Number of bottleneck reactions (tight bounds): {len(bottlenecks)}")
print(f"{'Reaction ID':<20} | {'dir':<5} | {'flux':>10} | {'lower':>10} | {'upper':>10} | expr_constrained")
print("-"*90)
for b in bottlenecks:
    print(f"{b['id']:<20} | {b['direction']:<5} | {b['flux']:>10.6f} | {b['lower']:>10.3f} | {b['upper']:>10.3f} | {b['expr_constrained']}")



Number of bottleneck reactions (tight bounds): 7
Reaction ID          | dir   |       flux |      lower |      upper | expr_constrained
------------------------------------------------------------------------------------------
PFL                  | upper |   1.000000 |      0.000 |      1.000 | True
ACALD                | lower |  -1.160000 |     -1.160 |      1.160 | True
ACKr                 | lower |  -1.190000 |     -1.190 |      1.190 | True
PYRt2                | lower |  -1.260000 |     -1.260 |      1.260 | True
THD2                 | upper |   4.500000 |      0.000 |      4.500 | True
LDH_D                | lower |  -2.070000 |     -2.070 |      2.070 | True
NADH16               | upper |  20.260000 |      0.000 |     20.260 | True


**Answer 4b**  
Bottlenecks are the reactions listed above whose flux equals a bound in the used direction.  
Those marked `expr_constrained=True` are bottlenecks specifically due to the imposed maximal reaction activities from Exercise 2.


In [16]:
zero_flux_constrained = []
for rid in expr_ids:
    if rid in model.reactions:
        f = float(solution.fluxes[rid])
        if abs(f) <= TOL:
            zero_flux_constrained.append(rid)

print("Constrained reactions with zero flux in the optimal solution:")
print(f"(Count: {len(zero_flux_constrained)})\n")
print(f"{'Reaction ID':<20} | Reaction name")
print("-"*60)
for rid in zero_flux_constrained:
    rxn = model.reactions.get_by_id(rid)
    print(f"{rid:<20} | {rxn.name}")

if zero_flux_constrained:
    chosen = zero_flux_constrained[0]  
    rxn = model.reactions.get_by_id(chosen)
    print("\nSelected reaction for explanation:", chosen, "-", rxn.name)
    print("Reaction equation:")
    print(rxn.reaction)
else:
    print("\n(No constrained zero-flux reactions found under current solution.)")


Constrained reactions with zero flux in the optimal solution:
(Count: 22)

Reaction ID          | Reaction name
------------------------------------------------------------
FORt2                | Formate transport in via proton symport
ME1                  | Malic enzyme (NAD)
ME2                  | Malic enzyme (NADP)
FRD7                 | Fumarate reductase
GLUN                 | Glutaminase
GLUt2r               | L glutamate transport via proton symport  reversible
ICL                  | Isocitrate lyase
FUMt2_2              | Fumarate transport via proton symport (2 H)
FBP                  | Fructose-bisphosphatase
ADK1                 | Adenylate kinase
MALS                 | Malate synthase
GLUSy                | Glutamate synthase (NADPH)
MALt2_2              | Malate transport via proton symport (2 H)
PPCK                 | Phosphoenolpyruvate carboxykinase
NADTRHD              | NAD transhydrogenase
ACALDt               | Acetaldehyde reversible transport
AKGt2r              

**Answer 4c**  
I select reaction **ICL**. Although it is constrained by a maximal reaction activity, its optimal flux is zero.  
**Network explanation (from the ESCHER core map):** the pathway containing **ICL** is not required for optimal biomass under the current bounds. Either (i) an upstream substrate is limiting, (ii) an alternative pathway supplies the product more efficiently, or (iii) the reaction’s forward direction does not align with the optimal metabolite flow. Therefore it remains unused at the optimum.
